In [ ]:
import numpy as np
import cv2
import os

In [ ]:
def extract_frames(video_path, output_dir, frame_skip=10):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    extracted_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_id % frame_skip == 0:
            frame_path = os.path.join(output_dir, f"frame_{extracted_frames:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            extracted_frames += 1
        frame_id += 1

    cap.release()
    print(f"Extracted {extracted_frames} frames to {output_dir}")
    return output_dir


def remove_background(video_path, output_dir):
    if not os.path.exists(output_dir):
      os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    background_subtractor = cv2.createBackgroundSubtractorMOG2()

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply background subtraction
        mask = background_subtractor.apply(frame)

        # Remove small noise and fill gaps
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        # Apply mask to the original frame
        object_frame = cv2.bitwise_and(frame, frame, mask=mask)

        x, y, w, h = cv2.boundingRect(mask)  # Bounding box
        cropped_frame = object_frame[y:y+h, x:x+w]



        # Save the processed frame
        output_path = f"{output_dir}/frame_{frame_idx:04d}.jpg"
        cv2.imwrite(output_path, cropped_frame)
        frame_idx += 1

    cap.release()

def color_threshold(video_path, output_dir, lower_hsv, upper_hsv):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create mask for the color range
        mask = cv2.inRange(hsv, lower_hsv, upper_hsv)

        # Apply mask to the original frame
        object_frame = cv2.bitwise_and(frame, frame, mask=mask)

        # Save the processed frame
        output_path = f"{output_dir}/frame_{frame_idx:04d}.jpg"
        cv2.imwrite(output_path, object_frame)
        frame_idx += 1

    cap.release()


In [ ]:

video_path = "/content/video.mp4" # Path
output_dir = "/content/frames"

# extract_frames(video_path, output_dir)
remove_background(video_path, output_dir)

# lower_hsv = np.array([10, 50, 50])
# upper_hsv = np.array([180, 150, 200])
# color_threshold(video_path, output_dir, lower_hsv, upper_hsv)
